In [1]:
import pandas as pd
import re
import os
from unicodedata import normalize
from bs4 import BeautifulSoup

In [2]:
input_1 = ''
input_2 = '4000 do 5000 PLN'
input_3 = '5.000 - 6.000 zł'
input_4 = 'from 10 000 to 15 000 PLN'


In [3]:
def parse_salary(salary) -> dict:
    salary = normalize('NFKD', salary) #znaki diakrytycznymi/ specjalne - zeby czytalo
    bounds = re.findall('[0-9]+', salary.replace(' ', ''))
    low_bound = bounds[0] if bounds else None
    high_bound = bounds[1] if len(bounds) > 1 else low_bound
    currency = salary.split()[-1] if salary.split() else None

    return {'low': low_bound,
            'high': high_bound,
            'currency': currency}

In [4]:
parse_salary(input_2)

{'low': '4000', 'high': '5000', 'currency': 'PLN'}

In [5]:
def parse_city(city) -> dict:
    if re.search('(Zdalnie)', city):
         _city = "Zdalna",
    else:
        _city = city

    return {'city': _city}

In [6]:
parse_city("Zdalnie")

{'city': ('Zdalna',)}

In [7]:
def parse_jobs(jobs) -> list:
    results = []

    for job in jobs:
        results.append(parse_job(job))

    return results

In [8]:
# ZANIM OPAKUJEMY W FUNKCJE

with open('../data/raw/data engineer_3.html', 'r', encoding='utf-8') as f:
    page_html = f.read()

page = BeautifulSoup(page_html, 'html.parser')
salary = page.find(class_='text-truncate badgy salary tw-btn tw-btn-secondary-outline tw-btn-xs ng-star-inserted')
salary_range = salary.text.strip()
print(salary_range)

location = page.find('nfj-posting-item-city')
if location is None:
    location = 'Not listed'
else:
    location = location.find('span')
    location = location.text.strip()
# location = parse_city(location)
print(location)

name = page.find('h3', class_='posting-title__position').text.strip()
print(name)

company = page.find('span', class_='d-block')
company = company.text.strip() if company else "Unknown"
print(company)

technology = page.find('common-posting-item-tag', class_='tw-btn-secondary-outline')
technology = technology.find('a', class_='ng-star-inserted').text.strip() if technology else 'N/A'

print(technology)

20 000  – 25 000  PLN
Zdalnie
Mid Data Engineer
Acaisoft Poland Sp. z.o.o
Python


In [9]:
def parse_job(job) -> dict:

    salary = job.find(class_='text-truncate badgy salary tw-btn tw-btn-secondary-outline tw-btn-xs ng-star-inserted')
    salary_range = salary.text.strip() if salary else None
    salary_data = parse_salary(salary_range)

    location = job.find('nfj-posting-item-city')
    if location is None:
        location = 'Not listed'
    else:
        location = location.find('span')
        location = location.text.strip()
    location = parse_city(location)

    name = job.find('h3', class_='posting-title__position').text.strip()

    company = job.find('span', class_='d-block')
    company = company.text.strip() if company else "Unknown"

    technology = job.find('common-posting-item-tag', class_='tw-btn-secondary-outline')
    technology = technology.find('a', class_='ng-star-inserted').text.strip() if technology else 'N/A'

    return {
        'location': location,
        'salary': salary_data,
        'name': name,
        'company': company,
        'technology': technology
    }

In [10]:
def get_data(page) -> list:
    results = []
# Wywołanie x.parent zwraca rodzica (bezpośredniego nadrzędnego elementu) dla tego elementu.
    jobs = [x.parent for x in page.find_all('div', class_='posting-image')]
    page_data = parse_jobs(jobs)
    results += page_data

    return results

In [12]:
# przyklad
with open('../data/raw/data engineer_1.html', 'r', encoding='utf-8') as f:
    page_html = f.read()

page = BeautifulSoup(page_html, 'html.parser')

results = get_data(page)
print(results)


[{'location': {'city': 'Warszawa'}, 'salary': {'low': '12000', 'high': '19000', 'currency': 'PLN'}, 'name': 'Data Engineer', 'company': 'Ramp Network', 'technology': 'BigQuery'}, {'location': {'city': ('Zdalna',)}, 'salary': {'low': '26000', 'high': '31000', 'currency': 'PLN'}, 'name': 'Senior Data Engineer', 'company': 'Acaisoft Poland Sp. z.o.o', 'technology': 'Python'}, {'location': {'city': ('Zdalna',)}, 'salary': {'low': '18000', 'high': '30000', 'currency': 'PLN'}, 'name': 'Big Data Engineering Tech Lead', 'company': 'P2 Recruitment', 'technology': 'Python'}, {'location': {'city': ('Zdalna',)}, 'salary': {'low': '20000', 'high': '32000', 'currency': 'PLN'}, 'name': 'Remote - Senior Azure Data Engineer', 'company': 'Infosys Consulting', 'technology': 'Azure'}, {'location': {'city': 'Warszawa'}, 'salary': {'low': '26880', 'high': '34440', 'currency': 'PLN'}, 'name': 'Databricks Data Engineer with Python', 'company': 'AXA IT Solutions', 'technology': 'Python'}, {'location': {'city':

In [ ]:
data_dir = '../data/raw'
results = []

for entry in os.scandir(data_dir):
    with open(
        os.path.join(data_dir, entry.name),
        encoding='UTF-8') as f:
            html = f.read()

    job = entry.name.split('_')[0]
    bs = BeautifulSoup(html)

    offers = get_data(bs)
    for offer in offers:
        offer['job'] = job
    results += offers

In [13]:
df = pd.json_normalize(results, sep='_')
df.head()

,name,company,technology,location_city,salary_low,salary_high,salary_currency
0,Data Engineer,Ramp Network,BigQuery,Warszawa,12000,19000,PLN
1,Senior Data Engineer,Acaisoft Poland Sp. z.o.o,Python,"(Zdalna,)",26000,31000,PLN
2,Big Data Engineering Tech Lead,P2 Recruitment,Python,"(Zdalna,)",18000,30000,PLN
3,Remote - Senior Azure Data Engineer,Infosys Consulting,Azure,"(Zdalna,)",20000,32000,PLN
4,Databricks Data Engineer with Python,AXA IT Solutions,Python,Warszawa,26880,34440,PLN


In [15]:
import datetime
# df.to_csv('../data/interim/job_offers.csv', sep=';', encoding='UTF-8', index=False)
df.to_csv(f'../data/interim/job_offers_{datetime.date.today()}.csv', sep=';', encoding='UTF-8', index=False)